In [1]:
import glob
import random
import numpy as np
from written_test_automation import pre, result, loc, config
import matplotlib.pyplot as plt

def s(im):
    plt.imshow(im,cmap="gray")
    plt.show()

# Net trainer

In [ ]:
folders_label =[("./answer_images/0/*",[1,0,0,0,0,0]),
                ("./answer_images/1/*",[0,1,0,0,0,0]),
                ("./answer_images/2/*",[0,0,1,0,0,0]),
                ("./answer_images/3/*",[0,0,0,1,0,0]),
                ("./answer_images/4/*",[0,0,0,0,1,0]),
                ("./answer_images/5/*",[0,0,0,0,0,1])]


In [ ]:
paths = []
labels = []
for path,label in folders_label:
    new_paths = glob.glob(path)
    labels = labels + ([label] * len(new_paths))
    paths = paths + new_paths

In [ ]:
x = []
for path in paths:
    x.append([pre.imread(path=path) / 255])
    
x = np.array(x, dtype = np.float64 )
y = np.array(labels, dtype = np.float64)

In [ ]:
x.shape

In [ ]:
import nets
net = nets.answer_model()
net.load_weights("./answer_weights.pickle")

In [ ]:
net.numpy_train(x,y, epochs=100)

In [ ]:
net.save_weights("answer_weights.pickle")

# Config finder

In [2]:

def pick(ims, to_print):
    ''' prints to file configs that worked
        threshold_kernel, blur_kernel, blob_size
    '''
    th_ker = random.uniform(0.00001,0.5)
    blur_kernel = random.uniform(0.00001,0.1)
    blob_size = random.uniform(0.00001,0.1)
    
    config.threshold_kernel_percent = th_ker
    config.median_blur_kernel_percent = blur_kernel
    config.blob_min_size_percent = blob_size
    print(config.threshold_kernel_percent, config.median_blur_kernel_percent, config.blob_min_size_percent)


    try:
        for im in ims:
            result.predict(im)
        print(to_print+" " + "yes ")
        return [th_ker,blur_kernel,blob_size]
    except:
        return None
def dir_pick(path, loops = 100):
    paths = glob.glob(path)
    ims = []
    for i in paths:
        print(i)
        ims.append(pre.imread(i))

        
    configs = []
    for i in range(loops):
        picked_config = pick(ims, str(i))
        
        if picked_config == None:
            pass
        else:
            configs = configs + [picked_config]
            
    return configs

In [6]:
con = dir_pick("./written_test_automation/im/*", 1000)

./written_test_automation/im/test6.jpg
./written_test_automation/im/test5.jpg
./written_test_automation/im/test1.jpg
./written_test_automation/im/test3.jpg
./written_test_automation/im/test4.jpg
./written_test_automation/im/test2.jpg
./written_test_automation/im/test0.jpg
0.04229481765561763 0.06732389278667733 0.0490399359617967
0.4397809876506784 0.0016628629954558796 0.04414443586612424
0.4886469787660735 0.06712064319636704 0.00623751556549547
0.3515723444516564 0.018016185601529892 0.0034758557697171905
0.0054350889781651975 0.008293890469988241 0.044468055478147764
0.2084116510730952 0.08014931604793396 0.0261043814077326
0.09338367344982405 0.09305630699073454 0.04669873356274489
0.43489775691829957 0.0673868184954535 0.06843523221014937
0.14570777081734826 0.02308669940093084 0.053084222552344136
0.19883778384451592 0.014284858426934801 0.08048274311852602
0.35361001436654166 0.00040624718909398194 0.0631856316682753
0.16853105091678963 0.07601478688665321 0.0729511350947841
0.

0.013406310548610505 0.06427906546401871 0.07716154107406839
0.10314392792662343 0.012947086709680589 0.028442774481280058
0.08209868766927968 0.09617443940645407 0.07740271362546787
0.05908446147641914 0.059915661005969394 0.013179857351900713
0.172560033575739 0.00786950816362562 0.06254416376868897
0.20350134252083446 0.06340698388456631 0.08732053836569302
0.018199149980389052 0.05118153134525236 0.06359087521300509
0.26083294715206184 0.08799261397322847 0.08279869287507487
0.045316108630826114 0.014847938573590111 0.056565023783546636
0.39220096303924334 0.033892341116158846 0.06959902642630926
0.4102118369230182 0.07581100043963844 0.06909963517638461
0.18702109852072446 0.045138067374591084 0.04255337026981686
0.018010136407076233 0.008031331256569145 0.059657231596629307
0.2071525712549674 0.02722904417937689 0.09554565505335062
0.1892075712211199 0.07685051785097345 0.07372111757453242
0.3561122533729077 0.06026363981563113 0.028841149294333703
0.47846122989584416 0.027859871

0.3664080007516427 0.04413704295337753 0.0851240836489405
0.17401332943099257 0.01396358065055962 0.01134646840103116
0.1459952602430948 0.0441581358289286 0.01651861740519038
0.407815554006943 0.05341952643333681 0.05741343106168161
0.4859501306824805 0.03623716261982303 0.04472245819495016
0.007309617531427276 0.060516976109429785 0.04308567379327263
0.16025185312269022 0.01844157261982651 0.08853053554932366
0.2034904081542013 0.03160620163123417 0.021228091810069405
0.1563037012808083 0.08575225985422583 0.04768759691778872
0.2886951655539175 0.09820688884012255 0.0007599279922133927
0.25935636194224476 0.09569351009944849 0.09920351141628357
0.43904705456079507 0.03704022261833273 0.08641192153091877
0.4225672634271755 0.014614675277417322 0.019495433071112074
0.10667048594611558 0.07331235408807979 0.08155532046011706
0.4938609253606374 0.04172286693631126 0.06647558670574369
0.45925265077059196 0.03924512810571711 0.02996539156662498
0.19759598656008956 0.04980187381750953 0.009

0.4267202430528023 0.055328994577381246 0.09107929798707173
0.2892895260248015 0.0009612649661441263 0.059747555388683996
0.25999885478995416 0.03698482616443883 0.03443400847356529
0.1754230059800486 0.021605685222404004 0.007917752446106312
0.005746608442771681 0.0033615401365917085 0.00033610627362640796
0.3925371498861391 0.04699957940516176 0.09649837512936911
0.041020273722332884 0.058174471575988145 0.07283857149697173
0.0019521913915962685 0.04615901514217612 0.019997775910787274
0.34915330065856653 0.04362554451751896 0.06535700323612659
0.3124858911640308 0.01462602070453877 0.09079459664616465
0.28675098437479035 0.08256251260650634 0.06571360730210686
0.46301972933506796 0.08801482803671795 0.06326305522188708


/home/av/my_prog/written_test_automation/loc.py:117: RuntimeWarning: divide by zero encountered in double_scalars
  degrees = np.degrees(np.arctan(sides[0] / sides[1]))


0.1486003850603142 0.015932910387681232 0.01781733187958668
0.433998055901021 0.039918390683517616 0.013457269222914953
0.16188773981244312 0.05778296117830041 0.0022104896358688926
0.36759048229926966 0.09055237450306793 0.03376818281429138
0.3969194022400102 0.008049801529816245 0.0005037686952786191
0.36699867878279785 0.07957618520057869 0.05011520435724764
0.3827863614794022 0.004832126988306412 0.03662635826430514
430 yes 
0.23040610922804314 0.05317515986952914 0.0675691613838506
0.09530246937516439 0.04327362399883743 0.06671446739459146
0.001060397479250245 0.08647808092280783 0.00851612449446961
0.11914024483785866 0.05759913087242927 0.036255986525204335
0.06682597085384406 0.01963324450712557 0.07973923466560215
0.38952410107440033 0.025434233402326763 0.019154749609303603
0.0010635922116813625 0.06797833214449235 0.08788990659693274
0.17145796623644732 0.06872439004450674 0.09298244227649656
0.02967196821612928 0.07707970833647171 0.007591607192909142
0.10389547973927951 0

0.040437545913305284 0.07558930284657055 0.02344091906758929
0.2883796921015736 0.0791341259638597 0.01895450377394487
0.1989183863596729 0.004877939944262377 0.05515066858572814
0.16192392426880783 0.08242937997084017 0.0735842508287478
0.008332111945092263 0.06116810401420608 0.015883275057318298
0.2845957667883956 0.006298702286708102 0.045482917504689774
0.41997503558883825 0.011357207358822024 0.07750707061818443
0.47554347294713023 0.012290380402650923 0.011200706973762057
0.33400250888461575 0.026082356956163816 0.04101954435772602
0.2046258610860515 0.04543237747145162 0.02494940626257096
0.09839546190836343 0.06134298652240805 0.006590046675835476
0.025226596372457017 0.01724941540743801 0.04061857277210608
0.1824283411242597 0.0947412720536615 0.06277827500348702
0.0988903622195169 0.09601614427227115 0.0028706206254915426
0.1131903808331885 0.09262760613284676 0.008078675953103916
0.49908946117554415 0.07858893852374235 0.022516675637324294
0.06012066082327902 0.035631361209

0.15758796580820486 0.04615774962733899 0.07655721094979816
0.41209159393948697 0.05725597776556808 0.09059575005167982
0.0022438281816355853 0.00769055927818475 0.09802418535017281
0.18922055712661254 0.09457589970338247 0.017846499078622582
0.022205023393439376 0.034733982945977925 0.055884016489210105
0.36834354227012234 0.0035783100094965055 0.09142769358134421
0.1613689494405306 0.05926670246378606 0.04578045212498479
0.3937358210360592 0.0843256251491001 0.06323861085035323
0.27573459671949674 0.04979645933316581 0.00571692287556125
0.3805321638123058 0.01831774408528435 0.02187025463993316
0.1913358331715735 0.0718076648671263 0.021586820241759813
0.43518821559661813 0.06311010338883498 0.06978937610571481
0.465113459426269 0.08344950198113688 0.05749871979768061
0.42379827769698275 0.03227355598674763 0.076250593263973
0.14242721017663443 0.016524389650953934 0.004812869406003505
0.3559723621405004 0.03368529918633873 0.03299692663611022
0.19478621195174778 0.06642302451092723 

0.06890162406276672 0.018386317644390405 0.0029649405788855126
0.1831803972501119 0.08735051402005316 0.0942684898243375
0.42976007512133435 0.038960406611344055 0.07219264110297138
0.37306861972521055 0.0639346356699057 0.023078228231436026
0.43081362773063964 0.06911728795577825 0.004328002597735135
0.4644356623400587 0.09619315421215317 0.01943978980079569
0.1041153636021325 0.07738544456700774 0.08562061327387646
0.22308048599145044 0.06994933672703765 0.033935310321857065
0.1542498383141911 0.008409664723586804 0.0842172137850149
0.1551979623204735 0.06266057974251658 0.08903058763244079
0.23664533614298586 0.044947517579093606 0.08530949054424503
0.07855754435787024 0.05337246504890427 0.019603004475605913
0.42043951776183647 0.08453248107291715 0.09205941974152883
0.07270626106886005 0.031035133217534358 0.021445955878507297
0.3048053334212653 0.07275756559765124 0.07976828694004336
0.27674149498795875 0.0006128547579146416 0.053596300357189826
0.3749354614216782 0.0057158957783

0.09807186085447202 0.06205471805030461 0.09071269786133446
0.07764190113553197 0.06401957705370168 0.061235228933849445
0.18373140771029312 0.01717824208837731 0.06886356193120571
0.43154110939728746 0.05066064093760178 0.0020846020081203114
0.34275675472776607 0.02869179687555041 0.05034419081923158
0.06387474715486038 0.055864569703905104 0.01931835043000096
0.42194259813911883 0.0669093125737587 0.07909637118625888
0.3331756393971229 0.07125722690917839 0.0032428087484846443
0.10531894052269747 0.033082820626768455 0.013252835299607928
0.2793765933280257 0.011297680253668344 0.08906271078694637
0.07138927318844396 0.05489514461474546 0.013844648157976642
0.17854351865259852 0.02557778278596395 0.011990859065576618
0.16259523782131613 0.0027476993513197907 0.09434590011537762
0.2503883013525951 0.008393051590286449 0.006216015261043442
0.056024663969135205 0.07525004242102494 0.02427790628892462
0.3701422748031508 0.015939497947328168 0.08540795426239657
0.20252865207953458 0.039276

In [8]:
np.mean(con, axis=0)

array([0.208, 0.01 , 0.033])

In [7]:
con

[[0.21753679117668995, 0.01649325296936795, 0.035365674569633576],
 [0.06427916631779851, 0.027392326403807014, 0.027448001892387552],
 [0.2104508014709504, 0.005771134935491125, 0.025994583964155365],
 [0.29004107664667167, 0.005489945382541248, 0.03800905040126705],
 [0.20694243432015708, 0.0062287714769627, 0.03321115403425514],
 [0.2807852298414951, 0.014313436377741125, 0.03626001015569509],
 [0.15104613902167482, 0.020761475660123593, 0.026839424292653097],
 [0.1844371411451601, 0.007872922703234664, 0.03873005776114718],
 [0.3827863614794022, 0.004832126988306412, 0.03662635826430514],
 [0.21917907180900162, 0.002579657886373682, 0.03819739948414558],
 [0.07506794221221531, 0.0057946464219489685, 0.03603560467886711],
 [0.21088086076323068, 0.006146151895706732, 0.024771624015071558]]